In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import os
import tensorflow.keras.layers as layers
from keras import callbacks
# from tensorflow.python.keras.applications import imagenet_utils

In [3]:
dataset_path = os.path.abspath('/home/nimahsn/Documents/DMD')
train_path = os.path.join(dataset_path, 'train/')
test_path = os.path.join(dataset_path, 'test/')
validation_path = os.path.join(dataset_path, 'validation/')

In [4]:
image_gen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=keras.applications.mobilenet_v3.preprocess_input,
)

In [54]:
batch_size = 48
train_iter = image_gen.flow_from_directory(directory=train_path, target_size=(224, 224), batch_size=batch_size)
valid_iter = image_gen.flow_from_directory(directory=validation_path, target_size=(224, 224), batch_size=batch_size)
test_iter = image_gen.flow_from_directory(directory=test_path, target_size=(224, 224), batch_size=batch_size, shuffle=False)

Found 96755 images belonging to 6 classes.
Found 13192 images belonging to 6 classes.
Found 24329 images belonging to 6 classes.


In [6]:
import time
base_name = 'new_data'
def get_board_path(name: str = ""):
    return os.path.join(".", "tensorboard_"+base_name, time.strftime("%Y-%m-%d_%H:%M:%S", time.gmtime())+"_"+ name)

def get_checkpoint_path(name: str = ""):
    return os.path.join(".", "checkpoints_"+base_name, time.strftime("%Y-%m-%d_%H:%M:%S", time.gmtime())+"_"+ name)

In [7]:
def show_error(history, skip, filename):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    plt.plot(np.arange(skip, len(loss), 1), loss[skip:])
    plt.plot(np.arange(skip, len(loss), 1), val_loss[skip:])
    plt.title(filename) #'model train vs validation loss'
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='best')
    plt.savefig(filename + '.jpg',bbox_inches='tight', dpi=500)
    plt.show()

In [8]:
def get_callbacks(name: str, pat_stop, pat_plateau):
    tb_cb = keras.callbacks.TensorBoard(get_board_path(name))
    plateau_cb =keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=pat_plateau)
    checkpoints_cb = keras.callbacks.ModelCheckpoint(filepath=get_checkpoint_path(name), save_best_only=True)
    early_cb = keras.callbacks.EarlyStopping(patience=pat_stop, restore_best_weights=True, min_delta=1e-4)
    return [tb_cb, plateau_cb, checkpoints_cb, early_cb]

# Mobilenet V3 - Imagenet

In [9]:
mobilenet = keras.applications.MobileNetV3Small(include_top=False, input_shape=(224, 224, 3), weights='imagenet')

In [10]:
channel_axis = 1 if keras.backend.image_data_format() == 'channels_first' else -1
last_point_ch = 1024
dropout_rate = 0.2
classes = 6
classifier_activation = keras.activations.softmax

In [11]:
x = layers.GlobalAveragePooling2D()(mobilenet.output)
if channel_axis == 1:
    x = layers.Reshape((mobilenet.last_point_ch, 1, 1))(x)
else:
    x = layers.Reshape((1, 1, last_point_ch))(x)
if dropout_rate > 0:
    x = layers.Dropout(dropout_rate)(x)
x = layers.Conv2D(classes, kernel_size=1, padding='same', name='Logits')(x)
x = layers.Flatten()(x)
x = layers.Activation(activation=classifier_activation,
                      name='Predictions')(x)

model = keras.Model(inputs=mobilenet.input, outputs = x)

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv (Conv2D)                   (None, 112, 112, 16) 432         rescaling[0][0]                  
__________________________________________________________________________________________________
Conv/BatchNorm (BatchNormalizat (None, 112, 112, 16) 64          Conv[0][0]                       
______________________________________________________________________________________________

## Frozen - 15 Epochs

In [13]:
for layer in mobilenet.layers:
    layer.trainable = False

In [14]:
n_epochs = 15
name_frozen = 'mobilenetv3_frozen_6class'

In [15]:
optimizer = keras.optimizers.Adam(learning_rate=0.02)
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])

In [16]:
history = model.fit_generator(train_iter, steps_per_epoch=train_iter.samples // batch_size, 
    validation_data=valid_iter, validation_steps=valid_iter.samples//batch_size, 
    epochs=n_epochs, use_multiprocessing=True, workers=4, callbacks = get_callbacks(name_frozen, 8, 3))

/home/nimahsn/anaconda3/envs/tf-gpu/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
2015/2015 [==============================] - 382s 179ms/step - loss: 1.2019 - accuracy: 0.8101 - val_loss: 0.4953 - val_accuracy: 0.9363
INFO:tensorflow:Assets written to: ./checkpoints_new_data/2021-08-23_20:40:09_mobilenetv3_frozen_6class/assets
Epoch 2/15
2015/2015 [==============================] - 354s 175ms/step - loss: 0.8142 - accuracy: 0.8783 - val_loss: 0.4218 - val_accuracy: 0.9563
INFO:tensorflow:Assets written to: ./checkpoints_new_data/2021-08-23_20:40:09_mobilenetv3_frozen_6class/assets
Epoch 3/15
2015/2015 [==============================] - 358s 177ms/step - loss: 0.9472 - accuracy: 0.8813 - val_loss: 0.4928 - val_accuracy: 0.9383
Epoch 4/15
2015/2015 [==============================] - 358s 177ms/step - loss: 1.0308 - accuracy: 0.8868 - val_loss: 1.6096 - val_accuracy: 0.7549
Epoch 5/15
2015/2015 [==============================] - 370s 183ms/step - loss: 1.0650 - accuracy: 0.8904 - val_loss: 0.8123 - val_accuracy: 0.9500
Epoch 6/15
2015/2015 [================

2015/2015 [==============================] - 352s 174ms/step - loss: 0.3602 - accuracy: 0.9266 - val_loss: 0.1707 - val_accuracy: 0.9694
INFO:tensorflow:Assets written to: ./checkpoints_new_data/2021-08-23_20:40:09_mobilenetv3_frozen_6class/assets
Epoch 15/15
2015/2015 [==============================] - 343s 170ms/step - loss: 0.3624 - accuracy: 0.9263 - val_loss: 0.1561 - val_accuracy: 0.9707
INFO:tensorflow:Assets written to: ./checkpoints_new_data/2021-08-23_20:40:09_mobilenetv3_frozen_6class/assets


##  Frozen - 35 Epochs -  warm start 15

In [24]:
n_epochs = 35
name_frozen_e15 = 'mobilenetv3_frozen_6class_epoch15'

In [30]:
optimizer = keras.optimizers.Adam(learning_rate=0.0005)
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])

In [31]:
history_e15 = model.fit_generator(train_iter, steps_per_epoch=train_iter.samples // batch_size, 
    validation_data=valid_iter, validation_steps=valid_iter.samples//batch_size, 
    epochs=n_epochs, use_multiprocessing=True, workers=4, callbacks = get_callbacks(name_frozen_e15, 8, 3), initial_epoch=15)

/home/nimahsn/anaconda3/envs/tf-gpu/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 16/35
2015/2015 [==============================] - 343s 168ms/step - loss: 0.3635 - accuracy: 0.9245 - val_loss: 0.1897 - val_accuracy: 0.9674
INFO:tensorflow:Assets written to: ./checkpoints_new_data/2021-08-23_22:20:34_mobilenetv3_frozen_6class_epoch15/assets
Epoch 17/35
2015/2015 [==============================] - 348s 173ms/step - loss: 0.3495 - accuracy: 0.9246 - val_loss: 0.1505 - val_accuracy: 0.9701
INFO:tensorflow:Assets written to: ./checkpoints_new_data/2021-08-23_22:20:34_mobilenetv3_frozen_6class_epoch15/assets
Epoch 18/35
2015/2015 [==============================] - 346s 171ms/step - loss: 0.3549 - accuracy: 0.9226 - val_loss: 0.1548 - val_accuracy: 0.9699
Epoch 19/35
2015/2015 [==============================] - 359s 178ms/step - loss: 0.3416 - accuracy: 0.9259 - val_loss: 0.1614 - val_accuracy: 0.9688
Epoch 20/35
2015/2015 [==============================] - 351s 174ms/step - loss: 0.3378 - accuracy: 0.9259 - val_loss: 0.1683 - val_accuracy: 0.9682
Epoch 21/35
2015/

2015/2015 [==============================] - 350s 173ms/step - loss: 0.3203 - accuracy: 0.9289 - val_loss: 0.1506 - val_accuracy: 0.9699
Epoch 30/35
2015/2015 [==============================] - 349s 173ms/step - loss: 0.3080 - accuracy: 0.9291 - val_loss: 0.1498 - val_accuracy: 0.9701
Epoch 31/35
2015/2015 [==============================] - 350s 173ms/step - loss: 0.3147 - accuracy: 0.9287 - val_loss: 0.1498 - val_accuracy: 0.9703
Epoch 32/35
2015/2015 [==============================] - 342s 170ms/step - loss: 0.3226 - accuracy: 0.9276 - val_loss: 0.1499 - val_accuracy: 0.9702
Epoch 33/35
2015/2015 [==============================] - 341s 169ms/step - loss: 0.3296 - accuracy: 0.9263 - val_loss: 0.1502 - val_accuracy: 0.9701


In [32]:
model.save('./saved_models/mobilenet3_frozen_6class_35epochs')

INFO:tensorflow:Assets written to: ./saved_models/mobilenet3_frozen_6class_35epochs/assets


In [33]:
df_hist_1 = pd.DataFrame(history.history)
df_hist_2 = pd.DataFrame(history_e15.history)

In [34]:
df_hist_1.to_pickle("./saved_history/mobilenet3_frozen_6class_0_14.pkl")
df_hist_2.to_pickle("./saved_history/mobilenet3_frozen_6class_15_34.pkl")

## Unfronzen from Conv_1 - 20 Epochs - warm start

In [9]:
model = keras.models.load_model("./saved_models/mobilenet3_frozen_6class_35epochs")

In [10]:
# model.layers[-17].name
for layer in model.layers:
    layer.trainable = False
for layer in model.layers[-17:]:
    layer.trainable = True

In [12]:
name_unfrozen_e35 = "mobilenet3-unfrozen-conv_1-6class-e35"
n_epochs = 60

In [14]:
optimizer = keras.optimizers.Adam(learning_rate=0.002)
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])
history_trainable = model.fit_generator(train_iter, steps_per_epoch=train_iter.samples // batch_size, 
    validation_data=valid_iter, validation_steps=valid_iter.samples//batch_size, 
    epochs=n_epochs, callbacks = get_callbacks(name_unfrozen_e35, 10, 4),
    use_multiprocessing=True, workers=4, initial_epoch = 30)

/home/nimahsn/anaconda3/envs/tf-gpu/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 31/60
2015/2015 [==============================] - 415s 194ms/step - loss: 0.6171 - accuracy: 0.9399 - val_loss: 0.0791 - val_accuracy: 0.9843
INFO:tensorflow:Assets written to: ./checkpoints_new_data/2021-08-24_06:26:05_mobilenet3-unfrozen-conv_1-6class-e35/assets
Epoch 32/60
2015/2015 [==============================] - 376s 186ms/step - loss: 0.0577 - accuracy: 0.9854 - val_loss: 0.0458 - val_accuracy: 0.9881
INFO:tensorflow:Assets written to: ./checkpoints_new_data/2021-08-24_06:26:05_mobilenet3-unfrozen-conv_1-6class-e35/assets
Epoch 33/60
2015/2015 [==============================] - 377s 187ms/step - loss: 0.0388 - accuracy: 0.9897 - val_loss: 0.0617 - val_accuracy: 0.9844
Epoch 34/60
2015/2015 [==============================] - 376s 186ms/step - loss: 0.0321 - accuracy: 0.9907 - val_loss: 0.0422 - val_accuracy: 0.9887
INFO:tensorflow:Assets written to: ./checkpoints_new_data/2021-08-24_06:26:05_mobilenet3-unfrozen-conv_1-6class-e35/assets
Epoch 35/60
2015/2015 [============

Epoch 44/60
2015/2015 [==============================] - 369s 183ms/step - loss: 0.0010 - accuracy: 0.9998 - val_loss: 0.0087 - val_accuracy: 0.9979
INFO:tensorflow:Assets written to: ./checkpoints_new_data/2021-08-24_06:26:05_mobilenet3-unfrozen-conv_1-6class-e35/assets
Epoch 45/60
2015/2015 [==============================] - 372s 184ms/step - loss: 6.7756e-04 - accuracy: 0.9998 - val_loss: 0.0091 - val_accuracy: 0.9982
Epoch 46/60
2015/2015 [==============================] - 373s 185ms/step - loss: 4.2366e-04 - accuracy: 0.9999 - val_loss: 0.0093 - val_accuracy: 0.9982
Epoch 47/60
2015/2015 [==============================] - 371s 184ms/step - loss: 3.1307e-04 - accuracy: 0.9999 - val_loss: 0.0095 - val_accuracy: 0.9979
Epoch 48/60
2015/2015 [==============================] - 370s 183ms/step - loss: 1.9515e-04 - accuracy: 1.0000 - val_loss: 0.0094 - val_accuracy: 0.9981
Epoch 49/60
2015/2015 [==============================] - 368s 182ms/step - loss: 2.2705e-04 - accuracy: 1.0000 - val

Epoch 58/60
2015/2015 [==============================] - 371s 184ms/step - loss: 5.7550e-05 - accuracy: 1.0000 - val_loss: 0.0086 - val_accuracy: 0.9982
Epoch 59/60
2015/2015 [==============================] - 369s 183ms/step - loss: 4.5868e-05 - accuracy: 1.0000 - val_loss: 0.0086 - val_accuracy: 0.9982
Epoch 60/60
2015/2015 [==============================] - 371s 184ms/step - loss: 4.8653e-05 - accuracy: 1.0000 - val_loss: 0.0086 - val_accuracy: 0.9982


In [15]:
model.save('./saved_models/mobilenet3_unfrozen_conv1_6class_60epochs')

INFO:tensorflow:Assets written to: ./saved_models/mobilenet3_unfrozen_conv1_6class_60epochs/assets


In [16]:
df_temp = pd.DataFrame(history_trainable.history)

In [17]:
df_temp.to_pickle("./saved_history/mobilenet3_unfrozen_conv1_6class_35_59.pkl")

# Evaluate

In [9]:
model = keras.models.load_model('./saved_models/mobilenet3_unfrozen_conv1_6class_60epochs')

In [10]:
model.evaluate(test_iter)

507/507 [==============================] - 154s 225ms/step - loss: 0.6394 - accuracy: 0.9183


[0.639431893825531, 0.9182868003845215]

In [13]:
model_frozen = keras.models.load_model("./saved_models/mobilenet3_frozen_6class_35epochs")

In [14]:
model_frozen.evaluate(test_iter)

507/507 [==============================] - 68s 131ms/step - loss: 0.5605 - accuracy: 0.8809


[0.5604909062385559, 0.8808829188346863]

# real time

In [15]:
def __draw_label(img, text, pos, bg_color):
    font_face = cv2.FONT_HERSHEY_SIMPLEX
    scale = 1
    color = (0, 0, 0)
    thickness = cv2.FILLED
    margin = 2

    txt_size = cv2.getTextSize(text, font_face, scale, thickness)

    end_x = pos[0] + txt_size[0][0] + margin
    end_y = pos[1] - txt_size[0][1] - margin

    cv2.rectangle(img, pos, (end_x, end_y), bg_color, thickness)
    cv2.putText(img, text, pos, font_face, scale, color, 1, cv2.LINE_AA)

In [16]:
dict_classes = {
    0:'drinking',
    1:'makeup',
    2:'phone',
#     3:'phone_right',
#     4:'radio',
    3:'reach_back',
#     4:'reach_side',
    4:'safe',
    5:'talk'

}

In [17]:
import cv2;
# face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

video = cv2.VideoCapture(0)

while True:
    check, frame = video.read()
    # faces = face_cascade.detectMultiScale(frame,
    #                                       scaleFactor=1.1, minNeighbors=5);
    # for x,y,w,h in faces:
        # frame = cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 3);
        # frame = cv2.addText(frame, 'text', (0,0), 'Inter')
    img = tf.image.resize(frame, [224, 224])
    img = keras.applications.mobilenet_v3.preprocess_input(img)
    preds = model.predict(img[tf.newaxis, ...])
    index = np.argmax(preds[0])

    __draw_label(frame, dict_classes[index] + '\n' + str(preds[0][index]), (20,20), (255, 0, 0))    

    cv2.imshow('Face_Detector', frame);

    key = cv2.waitKey(1);

    if key == ord('q'):

        break;
    # while True:
    #     if key == ord('p'):
    #         break

video.release();
cv2.destroyAllWindows();